In [ ]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [ ]:
import re
import torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [ ]:
def split_poems(input_text):
  result = input_text.split('*****')
  return result

In [ ]:
with open('esenin.txt', 'r') as f:
  text = f.read()

In [ ]:
poems = split_poems(text)
poems[0]

'Вот уж вечер. Роса\nБлестит на крапиве.\nЯ стою у дороги,\nПрислонившись к иве.\n\nОт луны свет большой\nПрямо на нашу крышу.\nГде-то песнь соловья\nВдалеке я слышу.\n\nХорошо и тепло,\nКак зимой у печки.\nИ березы стоят,\nКак большие свечки.\n\nИ вдали за рекой,\nВидно, за опушкой,\nСонный сторож стучит\nМертвой колотушкой.'

In [ ]:
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write('\n'.join(poems) + '\n')

In [ ]:
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to('cuda:0')

In [ ]:
def generate(input_text):
  input_ids = tokenizer.encode(input_text, return_tensors="pt").to('cuda:0')

  model.eval()
  with torch.no_grad():
      out = model.generate(
          input_ids,
          do_sample=True,
          num_beams=3,
          temperature=1.5,
          top_p=2.2,
          max_length=100,
          repetition_penalty=1.2,
    )
  generated_text = list(map(tokenizer.decode, out))[0]
  print(generated_text)

In [ ]:
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=32)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
generate("Кто я? Что я? Только лишь мечтатель,")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Кто я? Что я? Только лишь мечтатель, но в душе романтик… — бормотал он вслух.

В дверь постучали.

— Войдите, — позвал он.

— Что вам угодно?

— Мне хотелось бы поговорить с вами.

Дверь отворилась.

— Проходите, — пригласил граф.

— Благодарю вас, — ответил гость.

— А вы знаете,


In [ ]:
training_args = TrainingArguments(
    output_dir="./finetuned",         # The output directory
    overwrite_output_dir=True,        # Overwrite the content of the output dir
    num_train_epochs=40,              # number of training epochs
    per_device_train_batch_size=20,   # batch size for training
    per_device_eval_batch_size=32,    # batch size for evaluation
    warmup_steps=9,                   # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=5
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(), lr=1e-5), None)
)

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=440, training_loss=2.85989296653054, metrics={'train_runtime': 1009.5923, 'train_samples_per_second': 45.286, 'train_steps_per_second': 0.436, 'total_flos': 2517417363898368.0, 'train_loss': 2.85989296653054, 'epoch': 37.93})

In [ ]:
model.save_pretrained('model_esenin')

In [ ]:
generate("Кто я? Что я? Только лишь мечтатель,")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Кто я? Что я? Только лишь мечтатель,
Уставивший в даль прекрасные очи.

Мне чудится, все в душе горит огнем,
И мне мерещится, что я сам в огне.
Такой уж я на земле странник и странник.
Глупое сердце, не бейся.

Не бейся, не бейся, я для тебя готов -
Я готов для тебя выть и выть волком.



In [ ]:
generate("Кто я?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Кто я? Кто я такой?
Оглянись вокруг -
В этом мире все люди звери!

Но среди них есть и поэты,
И есть художники, и певцы.
И таких немало на белом свете,
Только не счесть их во всей земле.

Все они стремятся в высь,
В эти дали, где звенит прибой,
И с высоты своей огромной
Им все равно, что внизу творится.

Я живу среди людей,



In [ ]:
generate(" Друг мой, друг мой,")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Друг мой, друг мой, что ты мелешь?

Ты, верно, пьян и думаешь о другом.
Или, может быть, я тебе приснился.
Пойми, ведь это всего лишь сон,
Пей, сколько хочешь, хоть до утра.

Пойми, ведь это всего лишь сон,
Это все мне только снится.
Пойми, ведь это всего лишь сон,
Это все мне только снится.

И теперь, когда


In [ ]:
generate("Зачем все это?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Зачем все это?
Отраву я пью горькую
И смотрю на луну.

Небо синее, месяц синий,
Поле голубее и светлей.
Дымится рожь на полях,
Зеленеет чернь в полях.

Голова кружится мне и звенит,
Пью я горькую и гляжу.
Полюбил я девушку простую,
С косою на плечах.

Я пойду с нею


In [ ]:
generate("Зачем нам анализ текстов?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Зачем нам анализ текстов?
Нам бы только сердцем верить,
Чтоб в трудный час не дрожать,
Не киснуть в тоске бесцельной
И не прятать от жизни взгляд.

Пусть она катится к чертям -
Наша жизнь! Пусть катится!
Утопить бы в вине тоску -
За одну эту малость.

Пусть катится! Ведь так заведено -
Каждый сам кузнец своего счастья.
Как же без этого? Ведь так
